### make sure to install
```
pip install ipympl
pip install nodejs
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install jupyter-matplotlib
```

credit: https://stackoverflow.com/a/56416229/11756613

In [1]:
%load_ext lab_black
%matplotlib widget

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

plt.style.use("dark_background")

In [2]:
# temporary autocompletion bug fix
%config Completer.use_jedi = False

In [9]:
def square(G, n1, n2):
    pos1 = G.nodes[n1]["pos"]
    pos2 = G.nodes[n2]["pos"]
    direction = pos2 - pos1
    assert np.isclose(1, abs(direction))

    rotation = np.exp(2 * np.pi * 1j / 4)
    pos3 = pos2 + direction * rotation
    pos4 = pos3 + direction * rotation ** 2

    existing_pos = nx.get_node_attributes(G, "pos")

    match = [node for node, pos in existing_pos.items() if np.isclose(pos, pos3)]
    if match:
        # node already exists in this position, so use it
        assert len(match) == 1
        n3 = match[0]
    else:
        # no node exists in this position, so create it
        n3 = len(G.nodes)  # assumes nodes are numbered 0,1,2,...
        G.add_node(n3, pos=pos3)

    match = [node for node, pos in existing_pos.items() if np.isclose(pos, pos4)]
    if match:
        # node already exists in this position, so use it
        assert len(match) == 1
        n4 = match[0]
    else:
        # no node exists in this position, so create it
        n4 = len(G.nodes)  # assumes nodes are numbered 0,1,2,...
        G.add_node(n4, pos=pos4)

    G.add_edge(n1, n2)
    G.add_edge(n2, n3)
    G.add_edge(n3, n4)
    G.add_edge(n4, n1)


def triangle(G, n1, n2):
    pos1 = G.nodes[n1]["pos"]
    pos2 = G.nodes[n2]["pos"]
    direction = pos2 - pos1
    assert np.isclose(1, abs(direction))

    rotation = np.exp(2 * np.pi * 1j / 3)
    pos3 = pos2 + direction * rotation

    existing_pos = nx.get_node_attributes(G, "pos")

    match = [node for node, pos in existing_pos.items() if np.isclose(pos, pos3)]
    if match:
        # node already exists in this position, so use it
        assert len(match) == 1
        n3 = match[0]
    else:
        # no node exists in this position, so create it
        n3 = len(G.nodes)  # assumes nodes are numbered 0,1,2,...
        G.add_node(n3, pos=pos3)

    G.add_edge(n1, n2)
    G.add_edge(n2, n3)
    G.add_edge(n3, n1)

In [22]:
# initialize graph
G = nx.Graph()

G.add_node(0, pos=0)
G.add_node(1, pos=1)
G.add_edge(0, 1)

In [23]:
fig, ax = plt.subplots()
fig.tight_layout()
fig.set_size_inches(10, 10)
size = 10
ax.set_xlim([-size, size])
ax.set_ylim([-size, size])

positions = nx.get_node_attributes(G, "pos")
# convert complex position to a 2D tuple
positions = {key: (value.real, value.imag) for key, value in positions.items()}
nx.draw_networkx(G, positions, ax, with_labels=False, node_size=0, edge_color="white")


def onclick(event):
    click_pos = event.xdata + event.ydata * 1j
    positions = nx.get_node_attributes(G, "pos")
    positions = list(positions.items())
    positions.sort(key=lambda pair: abs(pair[1] - click_pos))
    n1, pos1 = positions[0]
    n2, pos2 = positions[1]

    # find on which side the click was
    side = ((click_pos - pos1) / (pos2 - pos1)).imag
    if 0 > side:
        # if it's the bad side, flip nodes
        n1, n2 = n2, n1

    if event.button == 1:  # left click
        triangle(G, n1, n2)
    elif event.button == 3:  # right click
        square(G, n1, n2)

    # convert complex position to a 2D tuple
    positions = nx.get_node_attributes(G, "pos")
    positions = {key: (value.real, value.imag) for key, value in positions.items()}
    nx.draw_networkx(
        G, positions, ax, with_labels=False, node_size=0, edge_color="white"
    )


cid = fig.canvas.mpl_connect("button_press_event", onclick)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
side

0.4105259676747431

In [11]:
B = G.copy()

In [8]:
B